# 報告 2022-09-02

## 元データの内訳

In [27]:
%%sh
cat ~/ABCT/comp-proto/Annotation-complete-IDed/BCCWJ-ABC*.psd | munge-trees -w > /tmp/comp-yori.psd
cat ~/ABCT/comp-proto/Annotation-complete-IDed/bccwj_kurabe_*.psd | munge-trees -w > /tmp/comp-kurabe.psd

In [28]:
# 「より」の文数
COUNT_YORI, *_ = ! cat /tmp/comp-yori.psd | wc -l
COUNT_YORI = int(COUNT_YORI)

# 「より」のうち，単文の数
# NOTE: CPに関しては，全て単文であることを目視済み。
# NOTE: tregex options: -s: one-liner, -w: whole tree
COUNT_YORI_SIMPLE, *_ = ! tregex -s -w '/^(VPm|VPsub|Sm|Ssub|CP)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_SIMPLE = int(COUNT_YORI_SIMPLE)

# 「より」のうち，連用節の数
COUNT_YORI_ADVERBIAL, *_ = ! tregex -s -w '/^(VPa|Sa)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_ADVERBIAL = int(COUNT_YORI_ADVERBIAL)

# 「より」のうち，連体節の数
COUNT_YORI_ADNOMINAL, *_ =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_ADNOMINAL = int(COUNT_YORI_ADNOMINAL)

# そもそも比較構文でない物の数
COUNT_YORI_NA, *_ = ! cat /tmp/comp-yori.psd | sed -e '/#comp/d' | wc -l
COUNT_YORI_NA = int(COUNT_YORI_NA)


In [29]:
# 「比べて」の文数
COUNT_KURABE, *_ = ! cat /tmp/comp-kurabe.psd | wc -l
COUNT_KURABE = int(COUNT_KURABE)

# 「比べて」のうち，単文の数
# NOTE: CPに関しては，全て単文であることを目視済み。
# NOTE: tregex options: -s: one-liner, -w: whole tree
COUNT_KURABE_SIMPLE, *_ = ! tregex -s -w '/^(VPm|VPsub|Sm|Ssub|CP)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_SIMPLE = int(COUNT_KURABE_SIMPLE)

# 「比べて」のうち，連用節の数
COUNT_KURABE_ADVERBIAL, *_ = ! tregex -s -w '/^(VPa|Sa)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_ADVERBIAL = int(COUNT_KURABE_ADVERBIAL)

# 「比べて」のうち，連体節の数
COUNT_KURABE_ADNOMINAL, *_ =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_ADNOMINAL = int(COUNT_KURABE_ADNOMINAL)

# そもそも比較構文でない物の数
COUNT_KURABE_NA, *_ = ! cat /tmp/comp-kurabe.psd | sed -e '/#comp/d' | wc -l
COUNT_KURABE_NA = int(COUNT_KURABE_NA)

In [30]:
# 集計
import pandas as pd

STAT = pd.DataFrame(
    {
        "全文数": [COUNT_YORI, COUNT_KURABE],
        "連用節数": [COUNT_YORI_ADVERBIAL, COUNT_KURABE_ADVERBIAL],
        "連体節数": [COUNT_YORI_ADNOMINAL, COUNT_KURABE_ADNOMINAL],
        "その他比較構文数": [
            COUNT_YORI - COUNT_YORI_ADVERBIAL - COUNT_YORI_ADNOMINAL - COUNT_YORI_NA,
            COUNT_KURABE - COUNT_KURABE_ADVERBIAL - COUNT_KURABE_ADNOMINAL - COUNT_KURABE_NA,
        ],
        "比較構文でない数": [COUNT_YORI_NA, COUNT_KURABE_NA],
    },
    index = ["より", "比べて"]
)

In [31]:
STAT

,全文数,連用節数,連体節数,その他比較構文数,比較構文でない数
より,2700,292,449,1112,847
比べて,1042,123,87,580,252


In [32]:
# 合計
STAT.sum()

全文数         3742
連用節数         415
連体節数         536
その他比較構文数    1692
比較構文でない数    1099
dtype: int64

## 学習データ／評価データ
9:1になるように，事前に分割した。

In [29]:
import datasets

# NOTE: private repoなので，事前にログインが必要。
ds = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    revision = "3846bb0dc229dcfa07857ed1ab8aa55d94066882",
    use_auth_token = True,
)

ds

Using custom data configuration abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735
Reusing dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 252.26it/s]


DatasetDict({
    train: Dataset({
        features: ['ID', 'tokens', 'comp'],
        num_rows: 3368
    })
    test: Dataset({
        features: ['ID', 'tokens', 'comp'],
        num_rows: 374
    })
})

* 学習データ数： 3,368文
* テストデータ数： 374文

### 学習データの例

#### 単文
```
5_BCCWJ-ABC-aa-simple
妻 が 仕事 に 精出す 一方 、 [[赤沼 は]cont [それ より]prej [もっと]diff [忙しい]deg 。]root

5_BCCWJ-ABC-aa-simple_predicted
[CLS] 妻 が 仕事 に 精 ##出す 一方 、 [赤 ##沼 は]cont [それ より]prej [もっと]diff [忙 ##しい]deg 。 [SEP]


95_bccwj_kurabe_text-aj-simplified
[[旧法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食生活 は [格段 に]diff [豊か]deg に なっ た 。]root

95_bccwj_kurabe_text-aj-simplified_predicted
[CLS] [旧 ##法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食 ##生活 は [格段 に]diff [豊か]deg に なっ た 。 [SEP]

35_bccwj_kurabe_text-ah-simplified
[[アカ ナマコ の 成長 は]cont [アオナマコ に]prej [比べ]prej [若干]diff [劣っ]deg て い た 。]root

35_bccwj_kurabe_text-ah-simplified_predicted
[CLS] [アカ ナ ##マコ の]cont 成長 は [アオ ##ナ ##マコ に 比べ]prej [若干]diff [劣っ]deg て い た 。 [SEP]
```

#### 連用節
```
23_bccwj_kurabe_text-af-simplified
ところが 、 一 九 八 五 年 九月 の プラザ 合意 以降 、 [[円 が]cont [ドル に 比べ て]prej [百 ％ 以上]diff [はね上がり]deg]root 、 突然 日本 は アメリカ より はるか に 高 コスト の 国 に なり まし た 。

23_bccwj_kurabe_text-af-simplified_predicted
[CLS] ところが 、 一 九 八 五 年 九 ##月 の プラザ 合意 以降 、 [円 が]cont [ドル に 比べ て]prej [百 % 以上]diff [はね ##上がり]deg 、 突然 [日本 は]cont [アメリカ より]prej [はるか に]diff [高 コスト]deg の 国 に なり まし た 。 [SEP]
```

#### 連体節
```
21_BCCWJ-ABC-as-simple
三 十 歳 の サラリーマン が [自分 より]prej [七]diff [、]diff [[[八 歳]diff [年下]deg]cont]root の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。

21_BCCWJ-ABC-as-simple_predicted
[CLS] 三 十 歳 の サラリーマン [が]cont [自分 より]prej [七 、 八 歳]diff [年下]deg の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。 [SEP]
```
（ `[が]cont` が変）

```
99_BCCWJ-ABC-au-simple
何 畳 ある か わから ない くらい 、 [[[教室 より も]prej [広い]deg]cont]root 部屋 。

99_BCCWJ-ABC-au-simple_predicted
[CLS] 何 畳 ある か わから ない くらい 、 [教室 より も]prej [広い]deg 部屋 。 [SEP]
```

## モデルの設定
* 使用した事前学習モデル： https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking
* このモデルの上で，NER（固有表現認識）の一種として，与えられた文のどのspanが，比較構文の要素に相当するのかについてのモデルを構築。
* 比較構文の要素：
    * prej(acent)：「より」句
    * cont(rast)：比較対象
    * diff(erence)：差の表現
    * deg(ree)：程度表現
    * root：比較構文の最大スコープ（NERモデルにおいては取り除いた）
* 例： [ [太郎が]cont [花子よりも]prej [3cm]diff [高い]deg ]root ことは意外だった。

In [36]:
# 学習パラメータ

training_args = dict(
    # output_dir = str(output_path),

    # エポック数
    num_train_epochs = 27,

    # バッチのサイズ
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 128,

    # 学習率
    learning_rate = 5e-5,
    
    warmup_steps = 200,
    weight_decay = 0,
    # save_strategy = IntervalStrategy.STEPS,
    save_steps = 1000,
    do_eval = True,
    # evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = 109,
    include_inputs_for_metrics = True,

    # 乱数シード
    seed = 2630987289,

    # logging_dir = str(output_path / "logs"),
    logging_steps= 10,
)

## 学習結果

In [37]:
result = {"score_spanwise_details": {"prej": {"CORRECT": 234, "WRONG_SPAN": 36, "MISSING": 6, "SPURIOUS": 32, "WRONG_LABEL_SPAN": 3}, "diff": {"CORRECT": 64, "MISSING": 9, "SPURIOUS": 19, "WRONG_SPAN": 9, "WRONG_LABEL_SPAN": 1}, "deg": {"CORRECT": 222, "SPURIOUS": 56, "MISSING": 35, "WRONG_LABEL_SPAN": 3, "WRONG_SPAN": 17, "WRONG_LABEL": 4}, "cont": {"CORRECT": 85, "SPURIOUS": 68, "MISSING": 28, "WRONG_SPAN": 37, "WRONG_LABEL": 3, "WRONG_LABEL_SPAN": 2}}, "score_spanwise": {"prej": {"possible_entries": 279, "actual_entries": 305, "precision_strict": 0.7672131147540984, "recall_strict": 0.8387096774193549, "F1_strict": 0.863013698630137, "precision_partial": 0.8262295081967214, "recall_partial": 0.9032258064516129}, "diff": {"possible_entries": 83, "actual_entries": 93, "precision_strict": 0.6881720430107527, "recall_strict": 0.7710843373493976, "F1_strict": 0.7784090909090909, "precision_partial": 0.7365591397849462, "recall_partial": 0.8253012048192772}, "deg": {"possible_entries": 281, "actual_entries": 302, "precision_strict": 0.7350993377483444, "recall_strict": 0.7900355871886121, "F1_strict": 0.7907375643224699, "precision_partial": 0.7632450331125827, "recall_partial": 0.8202846975088968}, "cont": {"possible_entries": 155, "actual_entries": 195, "precision_strict": 0.4358974358974359, "recall_strict": 0.5483870967741935, "F1_strict": 0.5914285714285714, "precision_partial": 0.5307692307692308, "recall_partial": 0.667741935483871}}, "score_spanwise_F1_strict": 0.7558972313225674, "score_tokenwise": {"IGNORE": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0, "support": 0}, "O": {"precision": 0.9365750528541226, "recall": 0.8965961361545538, "f1-score": 0.9161496521902613, "support": 5435}, "B-deg": {"precision": 0.7777777777777778, "recall": 0.8321678321678322, "f1-score": 0.8040540540540541, "support": 286}, "B-prej": {"precision": 0.8372881355932204, "recall": 0.8260869565217391, "f1-score": 0.8316498316498316, "support": 299}, "B-cont": {"precision": 0.6368715083798883, "recall": 0.6909090909090909, "f1-score": 0.6627906976744187, "support": 165}, "B-diff": {"precision": 0.7526881720430108, "recall": 0.7865168539325843, "f1-score": 0.7692307692307693, "support": 89}, "I-deg": {"precision": 0.6951219512195121, "recall": 0.7307692307692307, "f1-score": 0.7125, "support": 78}, "I-prej": {"precision": 0.8465732087227414, "recall": 0.9394987035436474, "f1-score": 0.8906185989348627, "support": 1157}, "I-cont": {"precision": 0.6471518987341772, "recall": 0.7226148409893993, "f1-score": 0.6828046744574291, "support": 566}, "I-diff": {"precision": 0.7142857142857143, "recall": 0.7222222222222222, "f1-score": 0.7182320441988951, "support": 90}, "micro avg": {"precision": 0.8769136558481323, "recall": 0.8769136558481323, "f1-score": 0.8769136558481323, "support": 8165}, "macro avg": {"precision": 0.6844333419610165, "recall": 0.7147381867210301, "f1-score": 0.6988030322390523, "support": 8165}, "weighted avg": {"precision": 0.881742860881822, "recall": 0.8769136558481323, "f1-score": 0.8784870999440403, "support": 8165}}}

In [38]:
# データカウント
df_res_count = pd.DataFrame.from_dict(
    result["score_spanwise_details"],
    orient = "index",
)

df_res_count

,CORRECT,WRONG_SPAN,MISSING,SPURIOUS,WRONG_LABEL_SPAN,WRONG_LABEL
prej,234,36,6,32,3,NaN
diff,64,9,9,19,1,NaN
deg,222,17,35,56,3,4.0
cont,85,37,28,68,2,3.0


* CORRECT: ぴったり
* WRONG_SPAN: spanにずれがある。
* SPURIOUS: 正解データにないspanを予測してしまっている。
* MISSING: 正解データにあるspanを予測できていない。
* WRONG_LABEL_SPAN: spanにずれがあり，かつ，ラベルも間違っている。
* WRONG_LABEL: spanにずれはないが，ラベルが間違っている。

In [39]:
# 統計
df_res_stat = pd.DataFrame.from_dict(
    result["score_spanwise"],
    orient = "index",
)

df_res_stat

,possible_entries,actual_entries,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial
prej,279,305,0.767213,0.838710,0.863014,0.826230,0.903226
diff,83,93,0.688172,0.771084,0.778409,0.736559,0.825301
deg,281,302,0.735099,0.790036,0.790738,0.763245,0.820285
cont,155,195,0.435897,0.548387,0.591429,0.530769,0.667742


### 凡例

* `possible_entries`: 予測されたspanの数
    ```
    ct["CORRECT"] + ct["WRONG_SPAN"] + ct["WRONG_LABEL"] + ct["WRONG_LABEL_SPAN"] + ct["MISSING"]
    ```
* `actual_entires`: テストデータにあるspanの数
    ```
    res["possible_entries"] - ct["MISSING"] + ct["SPURIOUS"]
    ```
* `precision_strict`: 予測のうち，当たっているものの数
    ```
    ct["CORRECT"] / res["actual_entries"]
    ```
* `recall_strict`: テストデータにあるもののうち，予測されたspanの数
    ```
    ct["CORRECT"] / res["possible_entries"]
    ```
* `precision_partial`：strictよりも緩い。 WRONG_SPANを50%カウントに入れている。
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["actual_entries"]
    ```
* `recall_partial`
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["possible_entries"]
    ```
* F1はprecisionとrecallの調和平均

In [40]:
# F1_strictの単純平均。ラベルのカウントで重みづけることはしていない。
result["score_spanwise_F1_strict"]

0.7558972313225674

# 学習結果：連体節なしバージョン

In [39]:
# まず，連体節に該当する文のIDを抜き出す。

TREES_YORI_ADNOMINAL =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq
TREES_KURABE_ADNOMINAL =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq

TREES_ADNOMINAL = TREES_YORI_ADNOMINAL + TREES_KURABE_ADNOMINAL

import re
_RE_TREE_ID = re.compile(r"\(ID (?P<ID>[^)]+)\)")
IDs_ADNOMINAL = [
    # tree.group("ID")
    tree.group("ID")
    for tree in 
    filter(None, map(_RE_TREE_ID.search, TREES_ADNOMINAL))
]

In [43]:
from typing import Sequence

import numpy as np
import torch
import torch.utils.data

import datasets
from transformers import BertForTokenClassification
import evaluate

from abct_comp_ner_utils.train import convert_records_to_vectors,ID2LABEL_DETAILED, LABEL2ID, _get_tokenizer, _get_evaluator

model = BertForTokenClassification.from_pretrained("./../../comparative-NER-result_2022-08")

assert(isinstance(model, BertForTokenClassification))

dataset = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    use_auth_token = True,
    split = "test",
)
assert(isinstance(dataset, datasets.Dataset))

# 連体節を排除
dataset = dataset.filter(
    lambda x: x["ID"] not in IDs_ADNOMINAL
)

dataset = dataset.map(convert_records_to_vectors)

model.eval()

def _make_spans(
    input: Sequence | np.ndarray,
    pred: Sequence | np.ndarray
):
    result = {
        "start": [],
        "end": [],
        "label": [],
    }

    current_label = ID2LABEL_DETAILED[0][0]
    current_span_start: int = 0

    for loc, (input_id, label_id) in enumerate(zip(input, pred)):
        label = ID2LABEL_DETAILED[label_id][0]

        if input_id == 0:
            # reached padding
            break
        elif current_label != label:
            # label changed
            # conclude the old label
            if current_label not in ("IGNORE", "O"):
                result["start"].append(current_span_start)
                result["end"].append(loc)
                result["label"].append(current_label)
            else:
                pass

            # switch to new label
            current_label = label
            current_span_start = loc

    return result

def _decode(tokens):
    tokens_decoded = _get_tokenizer().batch_decode(
        [t for t in tokens if t != 0],
        skip_special_tokens = True,
    )

    return [t.replace(" ", "") for t in tokens_decoded]

def _predict(
    examples: datasets.arrow_dataset.Example | datasets.arrow_dataset.Batch
):
    examples["tokens_re"] = [
        _decode(entry) for entry in examples["input_ids"]
    ]

    predictions_raw = model.forward(
        input_ids = torch.tensor(examples["input_ids"]),
        attention_mask = torch.tensor(examples["attention_mask"]),
        token_type_ids = torch.tensor(examples["token_type_ids"]),
    ).logits
    match predictions_raw:
        case torch.Tensor():
            predictions: np.ndarray = predictions_raw.argmax(dim = 2).numpy()
        case np.ndarray():
            predictions: np.ndarray = predictions_raw.argmax(axis = 2)
        case _:
            raise TypeError
    examples["prediction"] = predictions


    examples["comp_predicted"] = [
        _make_spans(i, p)
        for i, p in zip(examples["input_ids"], predictions)
    ]
    
    return examples

dataset = dataset.map(
    _predict,
    batched = True,
    batch_size = 128,
)

_eval: evaluate.Metric = _get_evaluator(
    "../comparative-NER-metrics"
)

res_no_adnom = _eval._compute(
    predictions = dataset["prediction"],
    references = dataset["label_ids"],
    input_ids = dataset["input_ids"],
    special_ids = _get_tokenizer().all_special_ids,
    label2id = LABEL2ID,
    id2label_detailed = ID2LABEL_DETAILED,
)

Using custom data configuration abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735
Reusing dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-52992c08f197a676.arrow
Loading cached processed dataset at /home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c700223d3fe58d43.arrow
100%|██████████| 3/3 [00:21<00:00,  7.31s/ba]


In [50]:
# データカウント
df_res_count_no_adnom = pd.DataFrame.from_dict(
    res_no_adnom["score_spanwise_details"],
    orient = "index",
)

df_res_count_no_adnom

,CORRECT,WRONG_SPAN,SPURIOUS,MISSING,WRONG_LABEL_SPAN,WRONG_LABEL
prej,194,31,33,5,3,NaN
diff,48,7,16,12,1,NaN
deg,185,12,48,29,3,2.0
cont,79,39,66,36,3,1.0


In [51]:
df_res_stat_no_adnom = pd.DataFrame.from_dict(
    res_no_adnom["score_spanwise"],
    orient = "index",
)

df_res_stat_no_adnom

,possible_entries,actual_entries,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial
prej,233,261,0.743295,0.832618,0.848178,0.802682,0.899142
diff,68,72,0.666667,0.705882,0.735714,0.715278,0.757353
deg,231,250,0.740000,0.800866,0.794179,0.764000,0.826840
cont,158,188,0.420213,0.500000,0.569364,0.523936,0.623418


In [52]:
# F1_strictの単純平均。ラベルのカウントで重みづけることはしていない。
res_no_adnom["score_spanwise_F1_strict"]

0.7368588448486533